In [22]:
import numpy as np
import pandas as pd
#from datetime import datetime
import datetime
import time
import numba
import os,sys

def LoadData(InputDir):
    """"""
    ## load raw data
    data = {
        'tra': pd.read_csv('%s/air_visit_data.csv' % InputDir, parse_dates= ['visit_date']),
        'as': pd.read_csv('%s/air_store_info.csv' % InputDir),
        'hs': pd.read_csv('%s/hpg_store_info.csv' % InputDir),
        'ar': pd.read_csv('%s/air_reserve.csv' % InputDir, parse_dates= ['visit_datetime', 'reserve_datetime']),
        'hr': pd.read_csv('%s/hpg_reserve.csv' % InputDir, parse_dates= ['visit_datetime', 'reserve_datetime']),
        'id': pd.read_csv('%s/store_id_relation.csv' % InputDir),
        'tes': pd.read_csv('%s/sample_submission.csv' % InputDir),
        'hol': pd.read_csv('%s/date_info.csv' % InputDir, parse_dates=['calendar_date']).rename(columns={'calendar_date': 'visit_date'})
    }
    return data

@numba.jit
def ApplyDayoff(VisitCols, ReserveCols):
    """"""
    n = len(VisitCols)
    result = np.zeros((n, 1), dtype= 'int8')
    for i in range(n):
        result[i] = (VisitCols[i]- ReserveCols[i]).days
    return result

def ApplyDate(cols):
    """"""
    n = len(cols)
    result = np.zeros((n, 1), dtype= 'int8')
    for i in range(n):
        result[i] = cols[i].split(' ')[0]

In [34]:
mod2id = {'ar': 'air', 'hr': 'hpg'}
### load data set
InputDir = '../../data/raw'
DataSet = LoadData(InputDir)
DataSet['tra']['visit_date'] = DataSet['tra']['visit_date'].dt.date
DataSet['hol']['visit_date'] = DataSet['hol']['visit_date'].dt.date
date_info = DataSet['hol']
wkend_holidays = date_info.apply((lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5
air_visit_data = DataSet['tra']
visit_data = air_visit_data.merge(date_info, on='visit_date', how='left')
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)
wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.
print(visitors.head(10))

           air_store_id day_of_week  holiday_flg  visitors
0  air_00a91d42b08b08d9      Friday            0  3.583535
1  air_00a91d42b08b08d9      Monday            0  3.203625
2  air_00a91d42b08b08d9      Monday            1  3.091042
3  air_00a91d42b08b08d9    Saturday            0  2.524065
4  air_00a91d42b08b08d9      Sunday            0  1.098612
5  air_00a91d42b08b08d9    Thursday            0  3.475056
6  air_00a91d42b08b08d9     Tuesday            0  3.325868
7  air_00a91d42b08b08d9   Wednesday            0  3.353439
8  air_0164b9927d20bcc3      Friday            0  2.309302
9  air_0164b9927d20bcc3      Friday            1  2.761234


In [39]:
sample_submission = DataSet['tes'].copy()
sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['visit_date'] = pd.to_datetime(sample_submission.id.map(lambda x: x.split('_')[2])).dt.date
sample_submission.drop(['visitors'], axis= 1, inplace= True)
sample_submission = sample_submission.merge(date_info, on='visit_date', how='left')
sample_submission = sample_submission.merge(visitors, on=['air_store_id', 'day_of_week', 'holiday_flg'], how='left')
print(sample_submission.head(20))

                                 id          air_store_id  visit_date  \
0   air_00a91d42b08b08d9_2017-04-23  air_00a91d42b08b08d9  2017-04-23   
1   air_00a91d42b08b08d9_2017-04-24  air_00a91d42b08b08d9  2017-04-24   
2   air_00a91d42b08b08d9_2017-04-25  air_00a91d42b08b08d9  2017-04-25   
3   air_00a91d42b08b08d9_2017-04-26  air_00a91d42b08b08d9  2017-04-26   
4   air_00a91d42b08b08d9_2017-04-27  air_00a91d42b08b08d9  2017-04-27   
5   air_00a91d42b08b08d9_2017-04-28  air_00a91d42b08b08d9  2017-04-28   
6   air_00a91d42b08b08d9_2017-04-29  air_00a91d42b08b08d9  2017-04-29   
7   air_00a91d42b08b08d9_2017-04-30  air_00a91d42b08b08d9  2017-04-30   
8   air_00a91d42b08b08d9_2017-05-01  air_00a91d42b08b08d9  2017-05-01   
9   air_00a91d42b08b08d9_2017-05-02  air_00a91d42b08b08d9  2017-05-02   
10  air_00a91d42b08b08d9_2017-05-03  air_00a91d42b08b08d9  2017-05-03   
11  air_00a91d42b08b08d9_2017-05-04  air_00a91d42b08b08d9  2017-05-04   
12  air_00a91d42b08b08d9_2017-05-05  air_00a91d42b0

In [43]:
missings = sample_submission.visitors.isnull()
## there are days that is holidays in test data set(2017/4/22-2017/5/31) while not in train data set(2016/1/1-2017/4/22)
## 'visitors' of which will be NaN, we can fill them with normal 'visitors' in the past DOW days.
## holiday_flg == 0 demonstrates normal days
print(len(sample_submission[sample_submission['visitors'].isnull() == True]))
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(visitors[visitors.holiday_flg==0], 
                                                                                on=('air_store_id', 'day_of_week'), 
                                                                                how='left'
                                                                               )['visitors_y'].values
print(len(sample_submission[sample_submission['visitors'].isnull() == True])) ## unfortunately length not changeed, it did nothing
## then we can try to fill them with mean 'visitors' in the past days.
missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values
print(len(sample_submission[sample_submission['visitors'].isnull() == True]))

448
448
0


In [2]:
mod2id = {'ar': 'air', 'hr': 'hpg'}
### load data set
InputDir = '../../data/raw'
DataSet = LoadData(InputDir)

### date related features
print('\n============')
for mod in ['tra', 'tes']:
    start0 = time.time()
    if (mod == 'tes'):
        DataSet[mod]['visit_date'] = DataSet[mod]['id'].map(lambda x: str(x).split('_')[2])
        DataSet[mod]['air_store_id'] = DataSet[mod]['id'].map(lambda x: '_'.join(x.split('_')[:2]))
        DataSet[mod]['visit_date'] = pd.to_datetime(DataSet[mod]['visit_date'])
    DataSet[mod]['dow'] = DataSet[mod]['visit_date'].dt.dayofweek
    DataSet[mod]['year'] = DataSet[mod]['visit_date'].dt.year
    DataSet[mod]['month'] = DataSet[mod]['visit_date'].dt.month
    DataSet[mod]['visit_date'] = DataSet[mod]['visit_date'].dt.date
    end0 = time.time()
    print('%s data: unique stores %s, total %s, time elased %.2fs.' %
            (mod, len(DataSet[mod]['air_store_id'].unique()), len(DataSet[mod]['air_store_id']), (end0 - start0)))
print('')
# for reservation data
for mod in ['hr', 'ar']:
    start1 = time.time()
    DataSet[mod]['visit_date'] = DataSet[mod]['visit_datetime'].dt.date
    DataSet[mod].drop(['visit_datetime'], axis= 1, inplace= True)
    DataSet[mod]['reserve_date'] = DataSet[mod]['reserve_datetime'].dt.date
    DataSet[mod].drop(['reserve_datetime'], axis= 1, inplace= True)
    end1 = time.time()
    print('time-consuming part %.2f.' % (end1 - start1))
end0 = time.time()
print('=============')
print('process date done, time consumed %.2f.\n' % (end0 - start0))


tra data: unique stores 829, total 252108, time elased 0.74s.
tes data: unique stores 821, total 32019, time elased 0.14s.

time-consuming part 10.90.
time-consuming part 0.50.
process date done, time consumed 11.54.



In [3]:
start0 = time.time()
def pop_std(x):
    return x.std(ddof=0)
for mod in ['ar', 'hr']:
    s0 = time.time()
    tmpdf = pd.DataFrame(data=ApplyDayoff(DataSet[mod]['visit_date'].values, DataSet[mod]['reserve_date'].values),
                             index=DataSet[mod].index, columns=['reserve_date_diff'])
    tmpdf = pd.concat([DataSet[mod], tmpdf], axis=1)
    e0 = time.time()
    s1 = time.time()
    tmpdf = tmpdf.groupby(['%s_store_id' % mod2id[mod], 'visit_date'], as_index=False).agg({'reserve_visitors': sum, 'reserve_date_diff': ['mean', 'median', pop_std]})
    tmpdf.columns = ['%s_store_id' % mod2id[mod], 
                   'visit_date', 
                   '%s_reserved_visitors_sum' % mod2id[mod], 
                   '%s_reserved_dayoff_mean' % mod2id[mod], 
                   '%s_reserved_visitors_median' % mod2id[mod], 
                   '%s_reserved_visitors_std' % mod2id[mod]
                  ]
    e1 = time.time()
    DataSet[mod] = tmpdf
    print(DataSet[mod].head(100))
    break

            air_store_id  visit_date  air_reserved_visitors_sum  \
0   air_00a91d42b08b08d9  2016-10-31                          2   
1   air_00a91d42b08b08d9  2016-12-05                          9   
2   air_00a91d42b08b08d9  2016-12-14                         18   
3   air_00a91d42b08b08d9  2016-12-17                          2   
4   air_00a91d42b08b08d9  2016-12-20                          4   
5   air_00a91d42b08b08d9  2017-02-18                          9   
6   air_00a91d42b08b08d9  2017-02-23                         12   
7   air_00a91d42b08b08d9  2017-03-01                          3   
8   air_00a91d42b08b08d9  2017-03-14                          4   
9   air_00a91d42b08b08d9  2017-03-21                          3   
10  air_00a91d42b08b08d9  2017-03-24                          2   
11  air_00a91d42b08b08d9  2017-04-04                          2   
12  air_0164b9927d20bcc3  2016-10-28                         12   
13  air_0164b9927d20bcc3  2016-11-01                         1

In [4]:
DataSet['hol'].head()

,visit_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [6]:
day_periods = (DataSet['tes']['visit_date'].max() - DataSet['tes']['visit_date'].min()).days

38

In [8]:
DataSet['tes']['visit_date'].min() - pd.to_timedelta(2,unit='d')

datetime.date(2017, 4, 21)

In [56]:
groupped = DataSet['tra'].groupby(['air_store_id', 'dow'])

In [57]:
groupped.get_group(('air_00a91d42b08b08d9', 0)).head()

,air_store_id,visit_date,visitors,dow,year,month,mean
87536,air_00a91d42b08b08d9,2016-07-04,20,0,2016,7,20.000000
87542,air_00a91d42b08b08d9,2016-07-11,25,0,2016,7,22.500000
87553,air_00a91d42b08b08d9,2016-07-25,16,0,2016,7,20.333333
87559,air_00a91d42b08b08d9,2016-08-01,16,0,2016,8,19.250000
87565,air_00a91d42b08b08d9,2016-08-08,27,0,2016,8,20.800000


In [69]:
records = []
for g in groupped.groups:
    tmp_dict = {}
    tmp_dict['id'] = g[0]
    tmp_dict['dow'] = g[1]
    tmp_dict['mean'] = groupped.get_group(g)['visitors'].mean()
    records.append(tmp_dict)
#print(records)
df = pd.DataFrame(data= records, index= range(len(records)))
print(df)

      dow                    id       mean
0       0  air_00a91d42b08b08d9  22.457143
1       1  air_00a91d42b08b08d9  24.350000
2       2  air_00a91d42b08b08d9  28.125000
3       3  air_00a91d42b08b08d9  29.868421
4       4  air_00a91d42b08b08d9  36.500000
5       5  air_00a91d42b08b08d9  14.973684
6       6  air_00a91d42b08b08d9   2.000000
7       0  air_0164b9927d20bcc3   7.500000
8       1  air_0164b9927d20bcc3   9.560000
9       2  air_0164b9927d20bcc3   9.678571
10      3  air_0164b9927d20bcc3   9.846154
11      4  air_0164b9927d20bcc3  11.464286
12      5  air_0164b9927d20bcc3   6.409091
13      0  air_0241aa3964b7f861   8.920635
14      1  air_0241aa3964b7f861   8.621212
15      2  air_0241aa3964b7f861   9.852941
16      3  air_0241aa3964b7f861   5.555556
17      4  air_0241aa3964b7f861   7.786885
18      5  air_0241aa3964b7f861  13.892308
19      6  air_0241aa3964b7f861  10.781250
20      0  air_0328696196e46f18   6.416667
21      1  air_0328696196e46f18   7.285714
22      2  

In [70]:
print(len(DataSet['tra'][DataSet['tra']['visitors'] == 0]))

0


In [64]:
tmp2 = tmp1.sort_values(by= ['visit_date'])
print(tmp2.head(10))

TypeError: sort_values() got an unexpected keyword argument 'by'

In [28]:
tmp2.rolling(3, min_periods= 1).mean()

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25.000000,2.0,2016.000000,1.000000
6,air_ba937bf13d40fb24,2016-01-20,28.000000,2.0,2016.000000,1.000000
12,air_ba937bf13d40fb24,2016-01-27,26.666667,2.0,2016.000000,1.000000
16,air_ba937bf13d40fb24,2016-02-03,24.333333,2.0,2016.000000,1.333333
22,air_ba937bf13d40fb24,2016-02-10,24.666667,2.0,2016.000000,1.666667
28,air_ba937bf13d40fb24,2016-02-17,22.333333,2.0,2016.000000,2.000000
34,air_ba937bf13d40fb24,2016-02-24,23.333333,2.0,2016.000000,2.000000
40,air_ba937bf13d40fb24,2016-03-02,19.666667,2.0,2016.000000,2.333333
46,air_ba937bf13d40fb24,2016-03-09,21.666667,2.0,2016.000000,2.666667
52,air_ba937bf13d40fb24,2016-03-16,23.000000,2.0,2016.000000,3.000000
